# Feasible mode enumeration vs Reachability cuts

In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.plot import plot_state_space_trajectory
from pympc.geometry.polyhedron import Polyhedron

In [ ]:
m = 1.
l = 1.
g = 10.
k = 100.
d = .1
h = .01

In [ ]:
# discretization method
method = 'explicit_euler'

# dynamics n.1 (free space)
A1 = np.array([[0., 1.],[g/l, 0.]])
B1 = np.array([[0.],[1/(m*l**2.)]])
S1 = LinearSystem.from_continuous(A1, B1, h, method)

# dynamics n.2 (right wall)
A2 = np.array([[0., 1.],[g/l-k/m, 0.]])
B2 = B1
c2 = np.array([0., k*d/(m*l)])
S2 = AffineSystem.from_continuous(A2, B2, c2, h, method)

# dynamics n.2 (left wall)
A3 = np.array([[0., 1.],[g/l-k/m, 0.]])
B3 = B1
c3 = np.array([0., -k*d/(m*l)])
S3 = AffineSystem.from_continuous(A3, B3, c3, h, method)

# list of dynamics
S = [S1, S2, S3]

In [ ]:
# state domain n.1
x1_min = np.array([-d/l, -1.])
x1_max = np.array([d/l, 1.])
X1 = Polyhedron.from_bounds(x1_min, x1_max)
assert X1.bounded

# state domain n.2
x2_min = np.array([d/l, -1.])
x2_max = np.array([2.*d/l, 1.])
X2 = Polyhedron.from_bounds(x2_min, x2_max)
assert X2.bounded

# state domain n.3
x3_min = np.array([-2.*d/l, -1.])
x3_max = np.array([-d/l, 1.])
X3 = Polyhedron.from_bounds(x3_min, x3_max)
assert X3.bounded

# input domain
u_min = np.array([-4.])
u_max = np.array([4.])
U = Polyhedron.from_bounds(u_min, u_max)
assert U.bounded

# domains
D = [Xi.cartesian_product(U) for Xi in [X1, X2, X3]]

In [ ]:
PWA = PieceWiseAffineSystem(S, D)

In [ ]:
def dynamics(x1, x2):
    x = np.array([x1, x2])
    return PWA.simulate(x, [np.zeros(PWA.nu)])[0][1]
fig = plt.figure()
n = 21
x1 = np.linspace(-2.*d/l, 2.*d/l, n)
x2 = np.linspace(-1., 1., n)
X1, X2 = np.meshgrid(x1, x2)
F1 = np.array([dynamics(i, j)[0] - i for i, j in zip(np.ravel(X1), np.ravel(X2))]).reshape(n, n)
F2 = np.array([dynamics(i, j)[1] - j for i, j in zip(np.ravel(X1), np.ravel(X2))]).reshape(n, n)
M = np.hypot(F1, F2)
Q = plt.quiver(X1, X2, F1, F2, M)
# plt.axis('equal')
# plt.xlabel(r'$x_1$')
# plt.ylabel(r'$x_2$')
x0 = np.array([-1., 0.])
# x_traj, ms_traj = PWA.simulate(x0, [np.zeros(2)]*100)
# plot_state_space_trajectory(x_traj)
plt.show()

In [ ]:
from pympc.control.hybrid_benchmark.utils import infeasible_mode_sequences
infeasible_mode_sequences(PWA,5)

In [ ]:
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

In [ ]:
# controller parameters
N = 30
Q = np.diag([1.,1.])
R = np.diag([1.])

# terminal set and cost
P = Q
X_N = Polyhedron.from_bounds(*[np.zeros(2)]*2)

In [ ]:
x_min = np.minimum.reduce([x1_min, x2_min, x3_min])
x_max = np.maximum.reduce([x1_max, x2_max, x3_max])

In [ ]:
# samples for the initial position and velocity of the ball
n_levels = 10
n_samples = 101
q_samples = np.linspace(x_min[0], x_max[0], n_samples)
v_samples = np.linspace(x_min[1], x_max[1], n_samples)
Q_samples, V_samples = np.meshgrid(q_samples, v_samples)

In [ ]:
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# get feasible sets (uses the feasible set of the CH method for the CHLC method)
controller = HybridModelPredictiveController(PWA, N, Q, R, P, X_N, method='Traditional formulation')
# controller.add_reachability_constraints(10)
cs = get_constraint_set(controller.prog)
print cs.A.shape
var_indices = {v: i for i, v in enumerate(controller.prog.getVars())}
assert var_indices[controller.prog.getVarByName('x0[0]')] == 0
assert var_indices[controller.prog.getVarByName('x0[1]')] == 1
sect = convex_hull_method_fast(cs, [0,1])

# samples on the grid
cost_mat = np.empty([n_samples]*2)
for i, q in enumerate(q_samples):
    for j, v in enumerate(v_samples):
        print(str(i) + ',' + str(j) + '   \r'),
        x0 = np.array([q, v])
        cost = controller.feedforward_relaxation(x0, [])[3]
        cost_mat[i,j] = cost

In [ ]:
from math import floor
def my_round(x):
    return floor(x*10.)/10.

levels = [my_round((i+1)*np.nanmax(cost_mat)/n_levels) for i in range(n_levels)]
cp = plt.contour(Q_samples, V_samples, cost_mat.T, levels=levels, cmap='viridis_r')
plt.colorbar(cp, label='Cost relaxed problem')
sect.plot(facecolor='w')
plt.grid(True)
plt.savefig('pendulum_2_walls_bm.pdf',bbox_inches='tight')
plt.show()